In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np
from tqdm import tqdm
import math

In [ ]:
transform = transforms.Compose([
                    transforms.ToTensor(), 
                    # transforms.Normalize((0.5,), (0.5,)),
            ])

In [ ]:
def CreateDataLoaders(Option, p_BatchSize):
    if Option == 1:
        train_dataset = datasets.MNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.MNIST("./", train=False, transform = transform, download=True)
    elif Option == 2:
        train_dataset = datasets.FashionMNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.FashionMNIST("./", train=False, transform = transform, download=True)
    
    elif Option == 5:
        train_dataset = datasets.CIFAR10("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.CIFAR10("./", train=False, transform = transform, download=True)
    
    elif Option == 6:
        train_dataset = datasets.CIFAR100("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.CIFAR100("./", train=False, transform = transform, download=True)

    
    Train_DataLoader = torch.utils.data.DataLoader(train_subset, batch_size = p_BatchSize, shuffle = True)
    Val_DataLoader = torch.utils.data.DataLoader(val_subset, batch_size = p_BatchSize, shuffle = True)
    Test_DataLoader = torch.utils.data.DataLoader(test_dataset, batch_size = p_BatchSize, shuffle = True)

    return Train_DataLoader, Val_DataLoader, Test_DataLoader

In [ ]:
class AllConv_IOCN(nn.Module):
    def __init__(self, OutputDim):
        super(AllConv_IOCN, self).__init__()
        # self.InputDim = InputDim
        self.OutputDim = OutputDim
        
        self.Conv1 = nn.Conv2d(3, 96, kernel_size = 3)
        self.Conv2 = nn.Conv2d(96, 96, kernel_size = 3)
        self.Conv3 = nn.Conv2d(96, 96, kernel_size = 3, stride = 2)
        self.Conv4 = nn.Conv2d(96, 192, kernel_size = 3)
        self.Conv5 = nn.Conv2d(192, 192, kernel_size = 3)
        self.Conv6 = nn.Conv2d(192, 192, kernel_size = 3, stride = 2)
        self.Conv7 = nn.Conv2d(192, 192, kernel_size = 3)
        self.Conv8 = nn.Conv2d(192, 192, kernel_size = 1)
        self.Conv9 = nn.Conv2d(192, self.OutputDim, kernel_size = 1)

        self.AvgPool = nn.AvgPool2d(kernel_size = 2) # In Paper it is mentioned to avg pool on 6 x 6

        self.ActFunc = nn.functional.relu
        # self.batch = nn.BatchNorm1d(800)
        # self.SftMax = nn.Softmax

    def forward(self, x):
        # print(x.shape)
        x = self.ActFunc(self.Conv1(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv2(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv3(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv4(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv5(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv6(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv7(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv8(x))
        # print(x.shape)
        x = self.ActFunc(self.Conv9(x))
        # print(x.shape)
        # print("Conv done")
        x = self.AvgPool(x)
        # print(x.shape)
        
        return x

In [ ]:
def TrainModel(p_model, loss_criteria, Optimizer, device,  p_TrainDL, Gamma = 5):
    TrainCorr = 0
    TotNumOfSamples = 0
    ValAccuracy = float('inf')

    for images, labels in tqdm(p_TrainDL):
        Optimizer.zero_grad()

        # images = images.reshape(images.shape[0], -1).to(device)
        images = images.to(device)
        labels = labels.to(device)

        pred = p_model(images)
        pred = pred.reshape(pred.shape[0], -1)
        # print(type(pred))
        # print(pred.shape)
        # print(pred)
        # pred=pred.to(torch.float)
        # labels=labels.to(torch.float)
        predClass = torch.max(pred.data, 1)[1]
        # predClass = np.argmax(pred, axis=-1)
        # predClass = pred.max(1)
        # print("Pred", predClass)
        # print(type(labels))
        # print("labels", labels)

        # predClass=predClass.to(torch.float)
        loss = loss_criteria(pred, labels)
        TrainCorr += (predClass==labels).sum()
        TotNumOfSamples += len(labels)
        # print("Correct", TrainCorr, " Out of", TotNumOfSamples)

        for name, param in p_model.named_parameters():
            # if "Linear1" in name or "bias" in name:
            if "Conv1" in name:
                continue
            tmpParam = param.data
            NewTmpParam = torch.exp(tmpParam - Gamma)
            tmpParam = torch.where(tmpParam<0, NewTmpParam, tmpParam)
            param.data = tmpParam

        loss.backward()
        Optimizer.step()
    
    return TrainCorr/TotNumOfSamples

def EvaluateModel(p_model, p_loader, device):
    Correct = 0
    TotalNumOfSamples = 0

    for images, labels in tqdm(p_loader):
        images = images.to(device)
        labels = labels.to(device)

        pred = p_model(images)
        pred = pred.reshape(pred.shape[0], -1)
        predClass = torch.max(pred.data, 1)[1]
        Correct += (predClass==labels).sum()
        TotalNumOfSamples += len(labels)
        # loss = loss_criteria(predClass, labels)
        # Accuracy += loss.cpu().data.item()

    return Correct/TotalNumOfSamples

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 100

In [ ]:
# -------------------------------------------- CIFAR-10 Dataset -----------------------------------------------------------
Option = 5
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_AllConv_IOCN_CIFAR10.pt"

model = AllConv_IOCN(10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)

    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)


print()

Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 118/118 [00:01<00:00, 86.30it/s]


EPOCH -  1 . Train Accuracy =  0.1038588210940361 , Validation Accuracy =  0.14239999651908875
Model Re-Saved


100%|██████████| 118/118 [00:01<00:00, 89.86it/s]


EPOCH -  2 . Train Accuracy =  0.10804706066846848 , Validation Accuracy =  0.10920000076293945


...

EPOCH -  100 . Train Accuracy =  0.20465882122516632 , Validation Accuracy =  0.1982666701078415



100%|██████████| 665/665 [00:08<00:00, 78.46it/s]


Train Accuracy =  0.21279999613761902


100%|██████████| 157/157 [00:01<00:00, 88.86it/s]

Test Accuracy =  0.21329998970031738
Generalization Gap =  -0.0004999935626983643


In [ ]:

model = AllConv_IOCN(10).to(device)
for name, param in model.named_parameters():
    print(name)

Conv1.weight
Conv1.bias
Conv2.weight
Conv2.bias
Conv3.weight
Conv3.bias
Conv4.weight
Conv4.bias
Conv5.weight
Conv5.bias
Conv6.weight
Conv6.bias
Conv7.weight
Conv7.bias
Conv8.weight
Conv8.bias
Conv9.weight
Conv9.bias
